In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
import pickle

%matplotlib inline

## 데이터 전처리

In [2]:
movie = pd.read_pickle('movie.pkl')
movie['고유코드'] = [str(x) + '_' + str(y) for (x, y) in zip(movie['영화명'], movie['감독'])]
movie = movie.loc[movie['개봉일'] >= '2015-01-01']
movie = movie.sort_values(['개봉일', '고유코드', '누적관객수'])
col_nm = ['개봉일', '종료일', '영화명','대표국적', '제작사', '배급사', '등급', '장르', '감독', '배우', '고유코드', '누적관객수', '스크린수']
movie = movie[col_nm]
movie['진행일'] = [int(len(pd.date_range(movie.iloc[i]['개봉일'], '20' + movie.iloc[i]['종료일']))) for i in range(len(movie))]
movie.loc[movie['진행일'] == 0] = None
movie = movie.drop(['종료일'], axis = 1)
movie.index = range(len(movie))

In [3]:
# 제작사 변수 사용이 어려워 제거
movie = movie.drop(['제작사'], axis = 1)

# 감독 정보가 없는 영화 제거
movie = movie.loc[pd.notnull(movie['감독'])]

# 배우 정보가 없는 영화 제거
movie = movie.loc[pd.notnull(movie['배우'])]

# 배급사 정보가 없는 영화 제거
movie = movie.loc[pd.notnull(movie['배급사'])]

In [4]:
# 공연, 성인물(에로) 제거
idx = []
for i in movie.index:
    if movie.loc[i, '장르'] not in ['공연', '성인물(에로)']:
        idx.append(i)
        
movie = movie.loc[idx]

In [5]:
# 개봉 월 정보 추가
movie['개봉시기'] = [str(int(v[5:7])) + '월' for v in movie['개봉일']]
movie = movie.drop(['개봉일'], axis = 1)

In [6]:
# 가장 마지막 표본 추출
movie = movie.drop_duplicates('고유코드', keep = 'last')
movie = movie.set_index('고유코드')

In [7]:
# 타겟 변수 생성
movie['평균관객수'] = [int(round(v, 0)) for v in movie['누적관객수'] / movie['진행일']]
movie = movie.drop(['진행일'], axis = 1)

In [8]:
# 결측값 확인
movie.isnull().sum(axis = 0)

영화명      0
대표국적     0
배급사      0
등급       0
장르       0
감독       0
배우       0
누적관객수    0
스크린수     0
개봉시기     0
평균관객수    0
dtype: int64

## 데이터 탐색

In [9]:
# 백만명을 달성한 배급사, 감독, 배우, 국적, 장르
million = movie.loc[movie['누적관객수'] >= 1000000, ['배급사', '감독', '배우', '대표국적', '장르']].drop_duplicates(keep = 'last')

In [10]:
print(len(million))
million.head()

126


,배급사,감독,배우,대표국적,장르
고유코드,,,,,
테이큰 3_올리비에 메가턴,이십세기폭스코리아(주),올리비에 메가턴,"리암 니슨,팜케 얀센,매기 그레이스,존 그리스,포레스트 휘테커",미국,"액션,범죄,스릴러"
박물관이 살아있다 : 비밀의 무덤_숀 레비,이십세기폭스코리아(주),숀 레비,"벤 스틸러,로빈 윌리엄스,댄 스티븐스,오웬 윌슨,벤 킹슬리,레벨 윌슨",미국,"어드벤처,코미디,가족"
오늘의 연애_박진표,씨제이이앤엠(주),박진표,"이승기,문채원,이서진,장진우,이경진,이동진,임하룡,이상협,김광규,김부선,가인,임종윤...",한국,"멜로/로맨스,코미디"
강남 1970_유하,(주)쇼박스,유하,"이민호,김래원,정진영,설현,유승목,한재영,이연두,정호빈,최진호,엄효섭,최병모,지대한...",한국,"액션,드라마"
"빅 히어로_돈 홀,크리스 윌리암스",월트디즈니컴퍼니코리아(주),"돈 홀,크리스 윌리암스","라이언 포터,스콧 애짓,다니엘 헤니,제이미 정,T.J. 밀러,제네시스 로드리게스,데...",미국,"애니메이션,액션,코미디"


In [11]:
A_lst = []
temp = [A_lst.extend(d.split(',')) for d in million['배급사']]
A_lst = [n for (n, v) in Counter(A_lst).most_common() if v >= 1]
print('백만명을 달성한 영화 배급사:', len(pd.unique(A_lst)))

B_lst = []
temp = [B_lst.extend(d.split(',')) for d in million['감독']]
B_lst = [n for (n, v) in Counter(B_lst).most_common() if v >= 1]
print('백만명을 달성한 영화 감독:', len(pd.unique(B_lst)))

C_lst = []
temp = [C_lst.extend(d.split(',')) for d in million['배우']]
C_lst = [n for (n, v) in Counter(C_lst).most_common() if v >= 1]
print('백만명을 달성한 영화 배우:', len(pd.unique(C_lst)))

D_lst = []
temp = [D_lst.extend(d.split(',')) for d in million['대표국적']]
D_lst = [n for (n, v) in Counter(D_lst).most_common() if v >= 1]
print('백만명을 달성한 국가:', len(pd.unique(D_lst)))

E_lst = []
temp = [E_lst.extend(d.split(',')) for d in million['장르']]
E_lst = [n for (n, v) in Counter(E_lst).most_common() if v >= 5]
print('백만명을 5번 이상 달성한 장르:', len(pd.unique(E_lst)))

백만명을 달성한 영화 배급사: 15
백만명을 달성한 영화 감독: 126
백만명을 달성한 영화 배우: 1195
백만명을 달성한 국가: 4
백만명을 5번 이상 달성한 장르: 12


In [12]:
lst = {'A': A_lst, 'B': B_lst, 'C': C_lst, 'D': D_lst, 'E': E_lst}
file = open('lst.pkl', 'wb')
pickle.dump(lst, file)
file.close()

In [12]:
# 더미변수 생성해야하는 변수 (개봉시기, 대표국적, 장르, 등급)

In [13]:
# 개봉시기 더미변수 생성
dummy = pd.DataFrame(index = movie.index)
lst = pd.unique(movie['개봉시기'])
for i in lst:
    col_n = '개봉_%s' % i
    dummy[col_n] = 0
    
    for j in dummy.index:
        if movie.loc[j, '개봉시기'] == i:
            dummy.loc[j, col_n] = 1

In [14]:
# 등급 더미변수 생성
lst = ['전체관람가', '12세이상관람가', '15세이상관람가', '청소년관람불가']
for i in lst:
    col_n = i
    dummy[col_n] = 0
    
    for j in dummy.index:
        temp = movie.loc[j, '등급'].split(',')
        if i in temp:
            dummy.loc[j, i] = 1

In [15]:
# 대표국적 더미변수 생성
lst = D_lst
for i in lst:
    col_n = i
    dummy[col_n] = 0
    
    for j in dummy.index:
        if movie.loc[j, '대표국적'] == i:
            dummy.loc[j, i] = 1

In [16]:
# 장르 더미변수 생성
lst = E_lst
for i in lst:
    col_n = i
    dummy[col_n] = 0
    
    for j in dummy.index:
        temp = movie.loc[j, '장르'].split(',')
        if i in temp:
            dummy.loc[j, i] = 1

In [17]:
# 배급사 더미변수 생성
dummy_ex = pd.DataFrame(index = movie.index)
lst = A_lst
for i in lst:
    col_n = i
    dummy_ex[col_n] = 0
    
    for j in dummy_ex.index:
        temp = movie.loc[j, '배급사'].split(',')
        if i in temp:
            dummy_ex.loc[j, i] = 1
            
# 감독 더미변수 생성
lst = B_lst
for i in lst:
    col_n = i
    dummy_ex[col_n] = 0
    
    for j in dummy_ex.index:
        temp = movie.loc[j, '감독'].split(',')
        if i in temp:
            dummy_ex.loc[j, i] = 1
            
# 배우 더미변수 생성
lst = C_lst
for i in lst:
    col_n = i
    dummy_ex[col_n] = 0
    
    for j in dummy_ex.index:
        temp = movie.loc[j, '배우'].split(',')
        if i in temp:
            dummy_ex.loc[j, i] = 1

In [18]:
dummy_ex['백만배급사'] = dummy_ex[A_lst].sum(axis = 1)
dummy_ex['백만감독'] = dummy_ex[B_lst].sum(axis = 1)
dummy_ex['백만배우'] = dummy_ex[C_lst].sum(axis = 1)
dummy_ex = dummy_ex[['백만배급사', '백만감독', '백만배우']]
#dummy_ex = (dummy_ex != 0).astype(int)

In [19]:
dummy = dummy.join(dummy_ex)

In [20]:
#scaler = MinMaxScaler()
#col_n = dummy.columns
#row_n = dummy.index
#dummy = scaler.fit_transform(dummy)
#dummy = pd.DataFrame(dummy, columns = col_n, index = row_n)

In [21]:
movie = dummy.join(movie[['평균관객수']])

In [22]:
# 데이터 분할
np.random.seed(1124142)
test_movie = np.random.choice(movie.index, 610, replace = False)

train = movie.drop(test_movie).sample(frac = 1)
test = movie.loc[test_movie].sample(frac = 1)

In [24]:
file = open('train_data.pkl', 'wb')
pickle.dump(train, file)
file.close()

file = open('test_data.pkl', 'wb')
pickle.dump(test, file)
file.close()